In [ ]:
pip install nba-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.6.2
    Uninstalling certifi-2024.6.2:
      Successfully uninstalled certifi-2024.6.2


In [ ]:
!pip install nba_api ipywidgets matplotlib pandas numpy scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00


In [ ]:
#proyecto final
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
import ipywidgets as widgets
from IPython.display import display, clear_output
from tqdm import tqdm

# Función para obtener la lista de jugadores activos
def get_players_list():
    players_info = players.get_active_players()
    players_dict = {player['full_name']: player['id'] for player in players_info}
    return players_dict

# Función para obtener estadísticas de un jugador
def get_player_stats(player_id):
    try:
        career = playercareerstats.PlayerCareerStats(player_id=player_id)
        df = career.get_data_frames()[0]
        # Filtrar solo las temporadas jugadas (excluir la temporada actual si está presente)
        df = df[df['SEASON_ID'] != 'Career']
        # Calcular porcentajes de tiro
        df['FG%'] = df['FG_PCT'] * 100
        df['FG3%'] = df['FG3_PCT'] * 100
        df['FT%'] = df['FT_PCT'] * 100
        # Calcular anotaciones por FGA
        df['Scored Per FGA'] = df['FGM'] / df['FGA']
        # Agregar la edad del jugador como una columna
        df['PLAYER_AGE'] = df['PLAYER_AGE'].astype(int)  # Convertir la edad a entero
        return df
    except Exception as e:
        print(f"Error al obtener estadísticas del jugador: {e}")
        return None

# Función para preparar los datos para la predicción
def prepare_data_for_prediction(df, stat):
    X_train = df[['PLAYER_AGE']].values.reshape(-1, 1)
    y_train = df[stat]
    return X_train, y_train

# Crear widgets de selección
players_dict = get_players_list()
player_dropdown = widgets.Dropdown(options=list(players_dict.keys()), description='Jugador:')
load_button = widgets.Button(description="Cargar Datos")
predict_button = widgets.Button(description="Predecir Estadísticas")
stat_dropdown = widgets.SelectMultiple(
    options=['PTS', 'AST', 'REB', 'STL', 'BLK', 'MIN', 'FGA', 'FG3A', 'FTA', 'FTM', 'FG%', 'FG3%','FG3M', 'FG3A', 'FG3_PCT', 'FT%','TOV', 'Scored Per FGA'],
    description='Estadísticas:'
)
output = widgets.Output()

def load_data(b):
    global df
    player_name = player_dropdown.value
    player_id = players_dict[player_name]
    df = get_player_stats(player_id)

    if df is not None:
        with output:
            clear_output()
            print(f"Datos de {player_name} cargados...\n")

            # Imprimir las columnas disponibles en el DataFrame
            print("Columnas disponibles:", df.columns.tolist())

            # Mostrar tabla de estadísticas
            display(df[['SEASON_ID', 'TEAM_ABBREVIATION', 'PTS', 'AST', 'REB', 'STL', 'BLK', 'MIN', 'FGA', 'FTM', 'FG3A', 'FTA',
                        'TOV','FG%', 'FG3%', 'FT%', 'FG3M', 'FG3A', 'FG3_PCT', 'Scored Per FGA']])

            # Calcular y mostrar el promedio de las estadísticas seleccionadas
            avg_stats = df[['PTS', 'AST', 'REB', 'STL', 'BLK', 'MIN',]].mean()
            print("\nPromedio de estadísticas:")
            for stat, value in avg_stats.items():
                print(f"{stat}: {value:.2f}")

            # Graficar estadísticas
            fig, axs = plt.subplots(1, 5, figsize=(20, 4))
            stats = ['PTS', 'AST', 'REB', 'STL', 'BLK']
            for i, stat in enumerate(stats):
                axs[i].plot(df['SEASON_ID'], df[stat], marker='o')
                axs[i].set_title(f'{stat}')
                axs[i].set_xlabel('Temporada')
                axs[i].set_ylabel(stat)
                axs[i].tick_params(axis='x', rotation=45)
            plt.show()
    else:
        with output:
            clear_output()
            print("No se pudieron cargar los datos del jugador.")

def predict_stats(b):
    if 'df' in globals():
        if '2022-23' not in df['SEASON_ID'].values:
            with output:
                clear_output()
                print("El jugador se ha retirado.\n")
            return

        predictions = {}
        improvements = {}
        selected_stats = stat_dropdown.value

        for stat in tqdm(selected_stats, desc="Prediciendo estadísticas"):
            X_train, y_train = prepare_data_for_prediction(df, stat)

            # Utilizar Random Forest para la predicción
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)

            next_season_prediction = model.predict([[df['PLAYER_AGE'].iloc[-1] + 1]])[0]
            last_season_value = y_train.iloc[-1]

            predictions[stat] = next_season_prediction
            improvements[stat] = ((next_season_prediction - last_season_value) / last_season_value) * 100

            # Evaluación del modelo
            y_pred = model.predict(X_train)
            mse = mean_squared_error(y_train, y_pred)
            print(f"Mean Squared Error for {stat}: {mse:.2f}")

        with output:
            clear_output()
            print(f"Predicción para la temporada {df['SEASON_ID'].iloc[-1][:4]}-{int(df['SEASON_ID'].iloc[-1][:4]) + 1}:\n")
            for stat in selected_stats:
                print(f"{stat}: {predictions[stat]:.2f}")

            print(f"\nPorcentaje de mejora para la próxima temporada:\n")
            for stat in selected_stats:
                print(f"{stat}: {improvements[stat]:.2f}%")

            overall_improvement = np.mean(list(improvements.values()))
            print(f"\nPorcentaje de mejora general para la próxima temporada: {overall_improvement:.2f}%")

            # Sugerencias de mejora
            print("\nSugerencias de mejora para la próxima temporada:")
            for stat, improvement in improvements.items():
                if improvement < 0:
                    if stat in ['PTS', 'FGA', 'FG3A', 'FTA'] :
                        print(f"Trabaja en tus {stat} para mejorar tu efectividad ofensiva.")
                    elif stat in ['AST']:
                        print(f"Practica tu manejo de balón y visión de juego para mejorar tus asistencias ({stat}).")
                    elif stat in ['REB']:
                        print(f"Enfócate en tus rebotes ({stat}) trabajando en tu posicionamiento y fuerza.")
                    elif stat in ['STL', 'BLK']:
                        print(f"Mejora tu defensa ({stat}) trabajando en tu anticipación y habilidades defensivas.")
                    elif stat in ['MIN']:
                        print(f"Aumenta tu tiempo en cancha ({stat}) trabajando en tu resistencia y consistencia.")
                    elif stat in ['FG%', 'FG3%', 'FT%', 'Scored Per FGA']:
                        print(f"Mejora tu precisión de tiro ({stat}) con más práctica y técnica de tiro.")

            # Graficar predicciones comparativas
            for stat in selected_stats:
                plt.figure(figsize=(10, 6))
                plt.plot(df['SEASON_ID'], df[stat], marker='o', label='Histórico')
                plt.plot(df['SEASON_ID'].iloc[-1][:4] + '-' + str(int(df['SEASON_ID'].iloc[-1][:4]) + 1),
                         predictions[stat], 'ro', label='Predicción')
                plt.title(f'Predicción de {stat}')
                plt.xlabel('Temporada')
                plt.ylabel(stat)
                plt.legend()
                plt.xticks(rotation=45)
                plt.show()

                # Comparar predicciones con valores históricos
                y_true = df[stat].values
                y_pred = model.predict(X_train)

                plt.figure(figsize=(10, 6))
                plt.plot(df['SEASON_ID'], y_true, marker='o', label='Valores Reales')
                plt.plot(df['SEASON_ID'], y_pred, marker='x', linestyle='--', color='r', label='Predicciones del Modelo')
                plt.title(f'Comparativa de {stat}: Valores Reales vs Predicciones del Modelo')
                plt.xlabel('Temporada')
                plt.ylabel(stat)
                plt.legend()
                plt.xticks(rotation=45)
                plt.show()

    else:
        with output:
            clear_output()
            print("Cargue los datos del jugador primero.\n")

# Asignar acciones a los botones
load_button.on_click(load_data)
predict_button.on_click(predict_stats)

# Mostrar widgets
display(player_dropdown, load_button, stat_dropdown, predict_button, output)

Dropdown(description='Jugador:', options=('Precious Achiuwa', 'Steven Adams', 'Bam Adebayo', 'Ochai Agbaji', '…

Button(description='Cargar Datos', style=ButtonStyle())

SelectMultiple(description='Estadísticas:', options=('PTS', 'AST', 'REB', 'STL', 'BLK', 'MIN', 'FGA', 'FG3A', …

Button(description='Predecir Estadísticas', style=ButtonStyle())

Output()

Prediciendo estadísticas: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]

Mean Squared Error for PTS: 4732.98
